<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/comment_bertopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install -U sentence-transformers
!pip install bertopic==0.14.1
!pip install bertopic[flair]
!pip install bertopic[gensim]
!pip install bertopic[spacy]
!pip install bertopic[use]
!pip install emoji
!pip install advertools
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-3.20.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.3
    Uninstalling protobuf-4.22.3:
      Successfully uninstalled protobuf-4.22.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensor

In [6]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel

import sklearn as sk
import os
import nltk
from nltk.data import find

import seaborn as sns
import matplotlib.pyplot as plt

import re
import string
import advertools as adv


In [7]:
df = pd.read_csv("comments.csv")
df

,Business Unit Name,EVP,Facility Type,District,Comment Language,Original Comment Text,English Comment,Compound Sentiment Score,Overall Sentiment,spl_char_chk,deEmojify_text,emoji_present_flag,all_punct_chk,english_comment_length
0,05R-Denver,"Griffith, Kelly",OF,60,en,I can step down and go back to being a pharmac...,I can step down and go back to being a pharmac...,-0.6908,Negative,False,I can step down and go back to being a pharmac...,False,False,277
1,05R-Denver,"Griffith, Kelly",OF,60,en,I have worked more hours than I would have lik...,I have worked more hours than I would have lik...,-0.4005,Negative,False,I have worked more hours than I would have lik...,False,False,278
2,05R-Denver,"Griffith, Kelly",OF,60,en,"Corporate SLT ""Thank You"" for implementing “Li...","Corporate SLT ""Thank You"" for implementing “Li...",-0.0258,Negative,False,"Corporate SLT ""Thank You"" for implementing “Li...",False,False,461
3,05R-Denver,"Griffith, Kelly",OF,60,en,We have made incredible strides in the Denver ...,We have made incredible strides in the Denver ...,0.0258,Positive,False,We have made incredible strides in the Denver ...,False,False,296
4,05R-Denver,"Griffith, Kelly",OF,60,en,"Work on benefits, seems poorer every year.","Work on benefits, seems poorer every year.",0.0258,Positive,False,"Work on benefits, seems poorer every year.",False,False,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58232,United,NaN,NaN,NaN,en,This is an amazing company to work with. In ou...,This is an amazing company to work with. In ou...,0.9947,Positive,False,This is an amazing company to work with. In ou...,False,False,1068
58233,United,NaN,NaN,NaN,en,"ever since I started working here, I have neve...","ever since I started working here, I have neve...",0.9949,Positive,False,"ever since I started working here, I have neve...",False,False,896
58234,United,NaN,NaN,NaN,en,Very good at working with request for days off...,Very good at working with request for days off...,0.9971,Positive,False,Very good at working with request for days off...,True,False,3213
58235,United,NaN,NaN,NaN,en,"I love working at my job it’s my happy place, ...","I love working at my job it’s my happy place, ...",0.9977,Positive,False,"I love working at my job it’s my happy place, ...",False,False,781


In [8]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2023-04-28 20:18:40,567 - BERTopic - Transformed documents to Embeddings
2023-04-28 20:19:21,974 - BERTopic - Reduced dimensionality
2023-04-28 20:20:05,939 - BERTopic - Clustered reduced embeddings


In [9]:
from bertopic import BERTopic
#from sklearn.feature_extraction.text import CountVectorizer
#topic_model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', vectorizer_model=None,min_topic_size= 15, nr_topics="auto", language = 'english',calculate_probabilities = True)
#topic_model=BERTopic()
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, nr_topics="auto")
comment_text = df['deEmojify_text'].values.tolist()
headline_topics, probs = topic_model.fit_transform(comment_text)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/1820 [00:00<?, ?it/s]

2023-04-28 20:21:07,510 - BERTopic - Transformed documents to Embeddings
2023-04-28 20:22:27,111 - BERTopic - Reduced dimensionality
2023-04-28 20:38:41,334 - BERTopic - Clustered reduced embeddings


In [10]:
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

Number of topics: 417


,Topic,Count,Name
0,-1,21788,-1_pay_company_more_and
1,0,6728,0_store_director_manager_he
2,1,2052,1_safeway_working_at_years
3,2,1031,2_albertsons_albertson_working_at
4,3,866,3_training_train_trained_new


In [11]:
a_topic = freq.iloc[1]["Topic"] # Select the 1st topic
topic_model.get_topic(a_topic) # Show the words and their c-TF-IDF scores

[('store', 0.00856787791148704),
 ('director', 0.006541356632705584),
 ('manager', 0.004902559499961339),
 ('he', 0.0046643543893757685),
 ('she', 0.004536160168502255),
 ('her', 0.003711336996204771),
 ('management', 0.00340118383693508),
 ('best', 0.0031631605764371852),
 ('stores', 0.003160164790320332),
 ('his', 0.0030181089942970104)]

In [12]:
topic_model.visualize_barchart(top_n_topics=10)

In [13]:
topic_model.visualize_topics()

In [14]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [15]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [16]:
similar_topics, similarity = topic_model.find_topics("pay", top_n = 5)
similar_topics 

[55, 155, 42, 39, 10]

In [17]:
freq[freq["Topic"].isin(similar_topics)]

,Topic,Count,Name
11,10,435,10_stores_pay_store_paid
40,39,158,39_paid_we_pay_deserve
43,42,149,42_here_paid_pay_working
56,55,125,55_employees_pay_should_heb
156,155,43,155_paycheck_pay_love_but


In [18]:
# Get the clusterer model, the clusters' tree and the clusters (topics ids)
clusterer = topic_model.hdbscan_model
tree = clusterer.condensed_tree_
clusters = tree._select_clusters()

# Get the ids of the most relevant documents (exemplars) associated with the topic at index idx
c_exemplars = topic_model.get_most_relevant_documents(clusters[idx], tree)

AttributeError: ignored

In [19]:
most_similar = similar_topics[0]
print("Most Similar Topic Info: \n{}".format(topic_model.get_topic(most_similar)))
print("Similarity Score: {}".format(similarity[0]))

Most Similar Topic Info: 
[('employees', 0.012337266581589833), ('pay', 0.011878901904362022), ('should', 0.011470114001438535), ('heb', 0.009718449083150307), ('wages', 0.008374303217969375), ('paid', 0.008243606966288813), ('better', 0.007924155075244872), ('more', 0.007604652412186037), ('lose', 0.006793385409691605), ('needs', 0.006781038277228902)]
Similarity Score: 0.771657055351289


In [21]:
topic_model.find_topics("management")

([99, 50, 179, 185, 244],
 [0.8051956839686121,
  0.7146434770761749,
  0.7001612996883935,
  0.6670831963382808,
  0.6619677534284839])

In [24]:
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model1 = BERTopic(hdbscan_model=hdbscan_model, language="english", calculate_probabilities=True, verbose=True, nr_topics="auto")
headline_topics1, probs1 = topic_model1.fit_transform(comment_text)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/1820 [00:00<?, ?it/s]

2023-04-28 20:47:05,844 - BERTopic - Transformed documents to Embeddings
2023-04-28 20:48:15,785 - BERTopic - Reduced dimensionality
2023-04-28 20:56:54,373 - BERTopic - Clustered reduced embeddings
2023-04-28 20:57:02,042 - BERTopic - Reduced number of topics from 333 to 234


In [25]:
topic_model1.visualize_barchart(top_n_topics=10)

In [26]:
topic_model1.visualize_topics()

In [ ]:
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model2 = BERTopic(hdbscan_model=hdbscan_model, language="english", calculate_probabilities=True, verbose=True, nr_topics="auto", ctfidf_model=ctfidf_model)
headline_topics2, probs2 = topic_model2.fit_transform(comment_text)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/1820 [00:00<?, ?it/s]

2023-04-28 21:04:07,374 - BERTopic - Transformed documents to Embeddings
